In [4]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import copy

# 1. Import Dataset

In [32]:
DATA_FOLDER_NAME = ''

data_folder = str(os.getcwd()) + "\data"
orig_filename = "/sample_orig_*.txt"


orig_headers = ['CREDIT_SCORE','FIRST_PAYMENT_DATE','FIRST_TIME_HOMEBUYER_FLAG','MATURITY_DATE','MSA','MI_PCT',
                'NUMBER_OF_UNITS','OCCUPANCY_STATUS','CLTV','DTI','ORIGINAL_UPB','LTV','ORIGINAL_INTEREST_RATE',
                'CHANNEL','PPM','AMORTIZATION_TYPE','PROPERTY_STATE', 'PROPERTY_TYPE','POSTAL_CODE',
                'LOAN_SEQUENCE_NUMBER','LOAN_PURPOSE', 'ORIGINAL_LOAN_TERM','NUMBER_OF_BORROWERS','SELLER_NAME',
                'SERVICER_NAME','SUPER_CONFORMING_FLAG','PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER', 
                'PROGRAM_INDICATOR', 'RELIEF_REFINANCE_INDICATOR', 'PROPERTY_VALUATION_METHOD', 'IO_INDICATOR']

raw_orig_types = {'CREDIT_SCORE':float, 'FIRST_PAYMENT_DATE':str, 'FIRST_TIME_HOMEBUYER_FLAG':str, 'MATURITY_DATE':str,
              'MSA':str, 'MI_PCT':float, 'NUMBER_OF_UNITS':float, 'OCCUPANCY_STATUS':str, 'CLTV':float, 'DTI':float,
              'ORIGINAL_UPB':float, 'LTV':float, 'ORIGINAL_INTEREST_RATE':float, 'CHANNEL':str, 'PPM': str,
              'AMORTIZATION_TYPE':str ,'PROPERTY_STATE':str, 'PROPERTY_TYPE':str ,'POSTAL_CODE':str,
              'LOAN_SEQUENCE_NUMBER':str, 'LOAN_PURPOSE':str, 'ORIGINAL_LOAN_TERM':float, 'NUMBER_OF_BORROWERS':float,
              'SELLER_NAME': str, 'SERVICER_NAME':str, 'SUPER_CONFORMING_FLAG':str,
              'PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER':str, 'PROGRAM_INDICATOR':str, 'RELIEF_REFINANCE_INDICATOR':str,
              'PROPERTY_VALUATION_METHOD':str, 'IO_INDICATOR':str}

mutated_orig_types = raw_orig_types.copy()

orig_dates = ['FIRST_PAYMENT_DATE', 'MATURITY_DATE']

data_folder = str(os.getcwd()) + "\data"
perf_filename = "\sample_svcg_*.txt"


perf_headers = ['LOAN_SEQUENCE_NUMBER','MONTHLY_REPORTING_PERIOD','CURRENT_ACTUAL_UPB',
                'CURRENT_LOAN_DELINQUENCY_STATUS','LOAN_AGE','REMAINING_MONTHS_TO_LEGAL_MATURITY', 
                'DEFECT_SETTLEMENT_DATE','MODIFICATION_FLAG', 'ZERO_BALANCE_CODE', 
                'ZERO_BALANCE_EFFECTIVE_DATE','CURRENT_INTEREST_RATE','CURRENT_DEFERRED_UPB','DDLPI',
                'MI_RECOVERIES', 'NET_SALE_PROCEEDS','NON_MI_RECOVERIES','EXPENSES', 'LEGAL_COSTS',
                'MAINTENANCE_AND_PRESERVATION_COSTS','TAXES_AND_INSURANCE','MISCELLANEOUS_EXPENSES',
                'ACTUAL_LOSS_CALCULATION', 'MODIFICATION_COST','STEP_MODIFICATION_FLAG','DEFERRED_PAYMENT_PLAN',
                'ELTV','ZERO_BALANCE_REMOVAL_UPB','DELINQUENT_ACCRUED_INTEREST','DELINQUENCY_DUE_TO_DISASTER',
                'BORROWER_ASSISTANCE_STATUS_CODE','CURRENT_MONTH_MODIFICATION_COST','INTEREST_BEARING_UPB']

raw_perf_types = {'LOAN_SEQUENCE_NUMBER':str, 'MONTHLY_REPORTING_PERIOD':str, 'CURRENT_ACTUAL_UPB':float,
              'CURRENT_LOAN_DELINQUENCY_STATUS': str, 'LOAN_AGE':float, 'REMAINING_MONTHS_TO_LEGAL_MATURITY':float, 
              'DEFECT_SETTLEMENT_DATE':str, 'MODIFICATION_FLAG':str, 'ZERO_BALANCE_CODE':str,
              'ZERO_BALANCE_EFFECTIVE_DATE':str, 'CURRENT_INTEREST_RATE':float, 'CURRENT_DEFERRED_UPB':float,
              'DDLPI': str,'MI_RECOVERIES':float, 'NET_SALE_PROCEEDS':str ,'NON_MI_RECOVERIES':float,'EXPENSES':float,
              'LEGAL_COSTS':float ,'MAINTENANCE_AND_PRESERVATION_COSTS':float, 'TAXES_AND_INSURANCE':float,
              'MISCELLANEOUS_EXPENSES':float,'ACTUAL_LOSS_CALCULATION':float, 'MODIFICATION_COST':float,
              'STEP_MODIFICATION_FLAG':str, 'DEFERRED_PAYMENT_PLAN':str, 'ELTV':float, 'ZERO_BALANCE_REMOVAL_UPB':float,
              'DELINQUENT_ACCRUED_INTEREST':float, 'DELINQUENCY_DUE_TO_DISASTER':str,'BORROWER_ASSISTANCE_STATUS_CODE':str,
              'CURRENT_MONTH_MODIFICATION_COST':float, 'INTEREST_BEARING_UPB':float}

mutated_perf_types = raw_perf_types.copy()
mutated_perf_types['CURRENT_LOAN_DELINQUENCY_STATUS'] = float



perf_dates = ['DEFECT_SETTLEMENT_DATE', 'ZERO_BALANCE_EFFECTIVE_DATE', 'DDLPI']

In [33]:
from FREDDIEMAC_preprocessing import pipeline_from_raw_data
import glob
from pathlib import Path


a = pipeline_from_raw_data(orig_filename, 
                       perf_filename,
                       orig_headers,
                       perf_headers,
                       raw_orig_types, 
                       raw_perf_types,
                       mutated_orig_types,
                       mutated_perf_types,
                       data_folder)

Working on sample_orig_2011 and sample_svcg_2011: : 2it [01:59, 59.64s/it]


In [ ]:
from FREDDIEMAC_preprocessing import create_origination_from_raw_data, create_performance_from_raw_data


df_orig = pd.read_csv(outputOrigFile, dtype=mutated_orig_types)
df_perf = pd.read_csv(outputPerfFile, dtype=mutated_perf_types)

In [ ]:
from FREDDIEMAC_classification import assign_labels_to_orig, assign_labels_to_perf, cutoff_sequence_according_to_label
from FREDDIEMAC_reducing import drop_short_sequences

df_orig = assign_labels_to_orig(df_orig, df_perf)
df_perf = assign_labels_to_perf(df_orig, df_perf)
df_orig, df_perf = cutoff_sequence_according_to_label(df_orig, df_perf)
df_orig, df_perf = drop_short_sequences(df_orig, df_perf, 12)

In [ ]:
df_orig.to_parquet("df_orig.parquet.gzip", compression='gzip')


In [ ]:
df_perf.to_csv("df_perf.csv", header=True, index=False)

In [ ]:
df_perf.to_parquet("df_perf.parquet.gzip", compression='gzip')

In [ ]:
df_orig = pd.read_csv(outputOrigFile, dtype=mutated_orig_types)
df_perf = pd.read_csv(outputPerfFile, dtype=mutated_perf_types)

In [ ]:
df_orig

In [ ]:
df_perf

In [ ]:
df_orig.columns

In [ ]:
df_perf.columns

In [ ]:
df_perf['LOAN_SEQUENCE_NUMBER'].unique()[:10]

In [ ]:
df_perf[df_perf['LOAN_SEQUENCE_NUMBER'] == "0000098"]["CURRENT_INTEREST_RATE"]

### 1.1 Data Exploration

In [ ]:
df_perf['LOAN_SEQUENCE_NUMBER'].nunique()

In [ ]:
df_orig['LOAN_SEQUENCE_NUMBER'].nunique()

# 2. Aggregating data like Blumenstock et al. 2020 experiment 4.1

In [ ]:
from FREDDIEMAC_reducing import reduce_length_of_sequence
from FREDDIEMAC_aggregation import aggregate_to_blumenstock_exp4

df_perf_concatenated = reduce_length_of_sequence(df_perf, length=12)
df_perf_concatenated.to_csv("df_perf_concatenated.csv", header=True, index=False)


In [ ]:
df_perf_concatenated = pd.read_csv("df_perf_concatenated.csv", dtype=mutated_perf_types)

In [ ]:
df_blumenstock = aggregate_to_blumenstock_exp4(df_orig, df_perf_concatenated)


In [ ]:
df_blumenstock.to_csv("df_blumenstock.csv", header=True, index=False)

In [ ]:
blumenstock_types = {'LOAN_SEQUENCE_NUMBER': str, 'ORIGINAL_INTEREST_RATE': float, 'ORIGINAL_UPB': float, 'CREDIT_SCORE': float,
                    'DTI': float, 'LTV': float, 'FIRST_PAYMENT_DATE': str, 'BAL_REPAID': float, 'T_ACT_12M': float, 'T_DEL_30D': float, 
                    'T_DEL_60D': float, 'LABEL': str, 'REMAINING_MONTHS_TO_LEGAL_MATURITY': float, 'TOTAL_OBSERVED_LENGTH': float}

df_blumenstock = pd.read_csv("df_blumenstock.csv", dtype=blumenstock_types)

# 3. Expanding Blumenstock et al. 2020 experiment 4.1 to dynamic data

In [1]:
import dask.dataframe as dd
df_perf = dd.read_parquet("df_perf.parquet.gzip")
df_orig = dd.read_parquet("df_orig.parquet.gzip")

In [2]:
from FREDDIEMAC_aggregation import aggregate_to_blumenstock_exp4_dynamic

df_dynamic_blumenstock = aggregate_to_blumenstock_exp4_dynamic(df_orig, df_perf)
#df_dynamic_blumenstock.to_parquet("df_dynamic_blumenstock.parquet.gzip", compression='gzip')

In [3]:
df_dynamic_blumenstock.head(10)

MemoryError: Unable to allocate 6.63 GiB for an array with shape (20, 44486097) and data type float64